# ArcGIS Online WAB Widget Audit

Description: This will audit all of the Web App Builder Aplications in an organization and the widgets within them.
 
Created on: 10/18/2023
 
Purpose: This will audit all of the web App Builder Applications in an organization,
    and provide a list of the widgets within them. You can then cross reference the list 
    of widgets with the following blog, in order to prioritize the order with which 
    you migrate your Web App Builder Applications to Experience Builder.
    https://community.esri.com/t5/arcgis-experience-builder-documents/functionality-matrix-for-web-appbuilder-and/ta-p/1113766
 
Authored By: Joe Guzi 

Inspired By: Brandon Wise
 
Previous Production Date: 2/5/24

Production Date: 8/12/24

Notes:
- 8/12/24 --- Added Audit Template.

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis.server import Server
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions

In [ ]:
# Functions
print("Begin Function Section.")
def ReturnWidgetData(Widget):
    global ItemID
    global ItemTitle
    global ItemOwner
    global ItemTags
    global ItemContentStatus
    global WABWA
    WidgetName = Widget.get("name")
    Visibility = Widget.get("visible")

    if WidgetName != None and Visibility != False:
        print(str(WidgetName) + "\n")
        WABWA += ItemID + ","
        WABWA += ItemTitle + ","
        WABWA += ItemOwner + ","
        WABWA += ItemTags + ","
        WABWA += ItemContentStatus + ","
        WABWA += WidgetName + ","
        WABWA += "\n"
        
def LoopThroughWidgetList(WidgetDataVariable):
    global ItemID
    global ItemTitle
    global ItemOwner
    global ItemTags
    global ItemContentStatus
    global WABWA
    WidgetList = WidgetDataVariable.get("widgets")
    if WidgetList != None:
        for Widget in WidgetList:
            #print(Widget)
            ReturnWidgetData(Widget)
    GroupList = WidgetData.get("groups")
    if GroupList != None:
        for Group in GroupList:
            WidgetList = Group.get("widgets")
            for Widget in WidgetList:
                #print(Widget)
                if WidgetList != None:
                    ReturnWidgetData(Widget)
            
print("Function Section Complete!")

# Step 4: Execute Code Section

In [ ]:
print("Begin Execute Code Section.") 
WebAppAudit = gis.content.search(query="", item_type="Web Mapping Application", max_items = 10000)
WABWA = "" # Web App Builder Widget Audit text

print("Begining to Loop Through Items" + "\n")
for Item in WebAppAudit:
    print("Entering Item Loop" + "\n")
    print(str(Item) + "\n")
    # The ArcGIS Knowledge Studio app on AGE is causing the Script to break... I'm bypassing it
    if Item.title != "ArcGIS Knowledge Studio":
        ItemID = str(Item.get("id"))
        TitelPrep = str(Item.get("title"))
        ItemTitle = TitelPrep.replace(',', "")
        ItemOwner = str(Item.get("owner"))
        TagPrep = str(Item.get("tags"))
        ItemTags = TagPrep.replace(',', "|")
        ItemContentStatus = str(Item.get("contentStatus"))

        JSON = Item.get_data(try_json=False) # when it breaks it is breaking here
        if JSON != None:
            ItemData = json.loads(JSON)
            #print("Item Data: " + str(ItemData)+ "\n")
            for ItemInfo in ItemData:
                # Operational Layers
                if ItemInfo == "widgetOnScreen":
                    print("Looping Through Widgets On Screen" + "\n")
                    WidgetData = ItemData.get("widgetOnScreen")
                    LoopThroughWidgetList(WidgetData)
                if ItemInfo == "widgetPool":
                    print("Looping Through Widget Pool" + "\n")
                    WidgetData = ItemData.get("widgetPool")
                    LoopThroughWidgetList(WidgetData)
    else:
        print("Ignore: ArcGIS Knowledge Studio")
        
print("Execute Code Section Complete!") 

# Step 5: Write the Results to Excel (Optional)

In [ ]:
print("Begin Write Results to Excel Section.") 
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("AppID,AppTitle,Owner,Tags,Status,Widget\n")
    output.writelines(WABWA)
    print("Wrtie data To CSV File Complete!")